In [3]:
#####################################   Importing data   #########################################################################################

import numpy as np
from vpython import *
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
from matplotlib.widgets import Slider, Button
from tkinter import *

###############################################################################################################################

<IPython.core.display.Javascript object>

In [4]:
########neutrino function########

# mixing factors (converted from degrees to radians)
th12 = np.radians(33.82)
th23 = np.radians(48.3)
th13 = np.radians(8.61)
delta_CP = np.radians(222)

# mass differences
dm21_sq = 7.53*10**(-5) # eV^2
dm32_sq = 2.52*10**(-3) # eV^2

# mass eigenstates
m1 = 0.75 # eV
m2 = np.sqrt(m1*m1 + dm21_sq) # eV
m3 = np.sqrt(m2*m2 + dm32_sq) # eV

#calculate third mass difference squared
dm31_sq = m3*m3 - m1*m1 # eV^2

# set energy values
E_sun_pp = 4*10**5 # eV 
E_atm=10**11 #eV
E_acc=10**9 #eV

# important constants:
# speed of light
c = 299792.458 # km s^-1 
# Planck constant
hbar = 6.582119569*10**(-16) # eV*s
# Astronomical unit (i.e. the distance between the Sun and the Earth)
AU = 149597870.700 # km



def prob_transition(alpha, beta, E, L):
    """Calculates the probability of a transition between an initial neutrino alpha to a neutrino beta
    Takes as inputs the initial and final neutrino flavours alpha and beta, 
    the energy at which the neutrino is produced E (in eV),
    and the distance travelled by the particle L (in km)
    Returns the probability of the transition"""
    
    if (alpha == "electron" and beta == "muon") or (alpha == "muon" and beta == "electron"): 
        ar = np.array([th12, dm21_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "electron" and beta == "tau") or (alpha == "tau" and beta == "electron"):
        ar = np.array([th13, dm31_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == "muon" and beta == "tau") or (alpha == "tau" and beta == "muon"):
        ar = np.array([th23, dm32_sq])
        arg = 1.27*ar[1]*L/E
        P = np.sin(2*ar[0])*np.sin(2*ar[0])*np.sin(arg)*np.sin(arg)
        
    elif (alpha == beta):
        
        if (alpha == "electron"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th13, dm31_sq])
            
        elif (alpha == "muon"):
            ar1 = np.array([th12, dm21_sq])
            ar2 = np.array([th23, dm32_sq])
            
        elif (alpha == "tau"):
            ar1 = np.array([th13, dm31_sq])
            ar2 = np.array([th23, dm32_sq])
        
        arg1 = 1.27*ar1[1]*L/E
        arg2 = 1.27*ar2[1]*L/E
        
        P1 = np.sin(2*ar1[0])*np.sin(2*ar1[0])*np.sin(arg1)*np.sin(arg1)
        P2 = np.sin(2*ar2[0])*np.sin(2*ar2[0])*np.sin(arg2)*np.sin(arg2)
        
        if ((P1 + P2) < 1) :
            P = 1 - (P1 + P2)
            
        else:
            P = 0
    
    else:
        return "Error"
    
    return P

In [11]:
####### Setting up initial data #######

g = 35               # gravitational acceleration (m.s^-2)
mass_bird = 1        # mass of the bird (kg)
radius_bird = 0.2    # radius of the bird (m)
x0 = 0.0             # initial x-coordinate of the bird
y0 = 0.5 + radius_bird # initial y-coordinate of the bird
p = 10              # density of the target (kg.m^-3) ~ density of a balsa type of wood
contact_time = 0.01  # contact time of the impact between the bird and the target (sec)
dt = 0.012           # time interval for the animations (sec)
d_angle = 0.06       # angle interval for the animation of the toppling of the target (rad)

bird_color = [color.blue,color.green,color.red]
bird_mass_topple = [0.3,1,1.8]

V_center = vector(8,0,0)
V_camera = vector(4,2,12)
V_title = vector(2,6,-2)
V_button = vector(-0.3,1,5)

In [12]:
####################### Functions ###########################

def trajectory(x_pos):
    '''
    Calculates the trajectory (y coordinate) at a given horizontal distance (x coordinate) from the lauch point.
    
    Input: 
     - x_pos: x coordinate of the point where the trajectory is required.
     
    Output:
     - trajectory: y coordinate of the particle at this point.
    '''
    
    trajectory = -g*(x_pos-x0)**2 / (2*v0**2*np.cos(theta)**2) + np.tan(theta)*(x_pos-x0) + y0
    
    return trajectory    



def vector_list():   
    global v0
    global theta
    Vectors=[]
    xpoints_slider=np.linspace(0,3,15)
    for x in xpoints_slider:
        Vectors.append(vector(x,trajectory(x),0))
    return Vectors

def setV(v):
    global v0
    wt1.text = 'Launch speed: ' + '{:1.0f}'.format(sl1.value) + ' m/s'
    v0 = v.value
    c.clear()
    c.append(vector_list())
    
def setTheta(t):
    global theta
    wt2.text = 'Launch angle: ' + '{:1.0f}'.format(sl2.value) + ' degrees'
    theta = np.radians(t.value)
    c.clear()
    c.append(vector_list())

In [13]:



######################### ANIMATION #####################

def runanim():
    
    global scene
    global v0
    global theta
    global target
    global c
    global level
    global label_level
    global target_pos
    
    # Initial data with respect to the scene set up 
    mass_target = p*(target.length*target.width*target.height)  # setting up the mass of the target (m = p*V)
    hit_tolerance = radius_bird + target.length/2               # computing the value of the hit tolerance : x-distance between bird.pos and target.pos at the time of the collision
    x_impact = target.pos.x - hit_tolerance                     # x-coordinate of the impact point
    t_restoring = mass_target*g*target.length/2                 # computing the magnitude of the restoring torque of the target

    
    
    
    obj = scene.mouse.pick
    
    if (obj == launch_button) :
        
        c.clear()

        x = x0
        y = y0
        t = 0
        distance_covered = 0

        # while the ball hasn't reached the target
        while x <= x_impact and y > 0 :

            # time increases by dt
            rate(25)
            t += dt

            distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

            prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*1.5*AU)
            prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*1.5*AU)
            prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*1.5*AU)

            prob = [prob_muon,prob_tau,prob_elec]
            prob_max = np.amax(prob)
            index_max = prob.index(prob_max)

            mass_topple = bird_mass_topple[index_max]
            bird.color = bird_color[index_max]
            bird.trail_color = bird_color[index_max]

            x = x0 + v0*t*np.cos(theta)               # new x-position of the bird
            y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
            bird.pos = vector(x,y,0)                  # updating the position of the bird


        # if the bird misses with the target
        if y > target.height + bird.radius/2 :

            # while the bird is above the ground 
            while x < 35 and y >= 0:

                # time increases by dt
                rate(25)
                t += dt

                distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

                prob_elec = prob_transition("muon", "electron", E_sun_pp , distance_covered*1.5*AU)
                prob_muon = prob_transition("muon", "muon", E_sun_pp , distance_covered*1.5*AU)
                prob_tau = prob_transition("muon", "tau", E_sun_pp , distance_covered*1.5*AU)

                prob = [prob_muon,prob_tau,prob_elec]
                prob_max = np.amax(prob)
                index_max = prob.index(prob_max)

                mass_topple = bird_mass_topple[index_max]
                bird.color = bird_color[index_max]
                bird.trail_color = bird_color[index_max]

                x = x0 + v0*t*np.cos(theta)               # new x-position of the bird
                y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
                bird.pos = vector(x,y,0)                  # updating the position of the bird
    

        # if the birds collides with target
        elif 0 < y < target.height + bird.radius/2 :

            c_rot = target.pos + vector(target.length/2,-target.height/2,0)   # center of rotation position
            bird_momentum = vector(mass_topple*v0*np.cos(theta) , mass_topple*v0*np.sin(theta) - mass_topple*g*t , 0) # momentum of the bird at the time of the impact
            vector_da = vector(x_impact,trajectory(x_impact),0) - c_rot       # vector from the point of rotation to the point of impact
            t_applied_vector = cross(bird_momentum/contact_time,vector_da)    # torque applied by the bird on the target
            t_applied = mag(t_applied_vector)                                 # magnitude of the torque applied 

            # if the applied torque is enough to topple the target 
            if t_applied > t_restoring :
                
                dtheta=0
                t=0
    
                while dtheta<=(np.pi/2):
                    rate(10)
                    dtheta+=(np.pi/2+0.001)/20
                    target.rotate(angle=(np.pi/2+0.001)/20,axis=vector(0,0,-1),origin=vector(target_pos+target.length/2,0,0))
                    
                    if y>bird.radius:
                        t += dt                                     # time increases by dt                    
                        y += -0.5*g*t**2                            # new y-position of the bird
                        bird.pos = vector(x,y,0)                    # updating the position of the bird
                
                level+=1
                target.height = 3+random()
                target_pos = (2*random()+1)*7
                target.pos = vector(target_pos,target.height/2,0)
                target.axis = vector(1,0,0)
                label_level.text= "Level "+str(level)
                 
                    
            # if the applied torque isn't enough to topple the target --> falling of the bird following a vertical straight line
            else :
                
                t = 0
                
                # while the bird is still above the ground 
                while y > bird.radius :
                    
                    rate(10)
                    t += dt                                     # time increases by dt                    
                    y += -0.5*g*t**2                            # new y-position of the bird
                    bird.pos = vector(x,y,0)                    # updating the position of the bird
                    
        
        # Initializing
        bird.clear_trail()
        bird.make_trail = False
        bird.pos = vector(x0,y0,0)
        bird.color = bird_color[0]
        mass_topple = bird_mass_topple[0]
        bird.make_trail = True
        
        c.append(vector_list())

#############################################################################################################################

In [38]:
################### CANVAS #######################

# Initial parameters
v0=10
theta=45
target_pos=(2*random()+1)*7
target_height=3 + random()
level = 1

# Setting up the scene 
scene = canvas(width=1000, height=700, center=V_center, range=8, background=color.cyan)#, texture = 'Clouds.jpg') # scene where the game animations will be displayed
scene.camera.pos = V_camera                                                               # position of the camera (from where the player sees the game)
scene.forward = V_center - scene.camera.pos                                               # axis of the camera (in which direction the player is looking)
scene.camera.axis = V_center - scene.camera.pos                                           # axis of the camera (in which direction the player is looking)
scene.userzoom = False
scene.userspin = False
    
# positioning the ground of the animation
ground = box(pos= V_center+vector(0,-0.01,0), length=100, height=0.01, width=100, texture = 'grass.jpg')

#cloud1 = box(pos= vector(10,5,-60), length=5, height=4, width=6, color=color.white)
#cloud2 = box(pos= vector(15,8,-45), length=4, height=7, width=2, color=color.white)
#cloud3 = box(pos= vector(35,14,-23), length=8, height=8, width=15, color=color.white)
#cloud4 = box(pos= vector(24,3,-56), length=13, height=5, width=4, color=color.white)
#cloud5 = box(pos= vector(22,7,-46), length=5, height=9, width=21, color=color.white)

# positioning the target at a random position
target = box(pos=vector(target_pos,target_height/2,0), length= 0.5, height=target_height , width=0.5, color = color.black)
             
# box
pedestal = box(pos = vector(0,0.25,0), length = 1, width = 2, height = 0.5, texture = textures.wood)

#sky
sky = box(pos=vector(50,22,-50),axis=vector(1,0,0.7), length=200, width=1, height=50,texture="Clouds.jpg")

# positioning the bird on the origin of the scene
bird = sphere(pos = vector(x0,y0,0), radius = 0.2, color=color.blue, make_trail = True, trail_color = color.blue)

# Text
label_title = label(text = 'ANGRY NEUTRINOS',pos = vector(8,7,0), color = color.black, box = False, background = color.cyan , height = 40, opacity = 0)
label_level = label(text = "Level "+str(level) , pos = vector(8,6,0), color = color.black, box = False, background = color.cyan , height = 35, opacity = 0)
      

# Launch button
launch_button = box(pos = V_button, length = 0.1, width = 0.3, height = 0.3, axis = V_button + V_camera + vector(-4,0,0), color = color.black)
launch_button.rotate(angle =- np.pi/6.5, axis= V_camera, origin=V_button)
label_button = label(text = 'GO',pos = V_button, color = color.white, box = False, background = color.black , height = 25, opacity = 1, border = 10)

# Aiming
c = points(radius = 3)  
c.append(vector_list())


# Slider Speed
sl1 = slider(min = 10 , max = 30 , value = 10, length = 220 , bind = setV, left = 50, right = 15)
wt1 = wtext(text = 'Launch speed: ' + '{:1.0f}'.format(sl1.value) + ' m/s') 

# Slider angle    
sl2 = slider(min = 0 , max = 90 , value = 45, length = 220 , bind = setTheta, left = 150, right = 15)
wt2 = wtext(text = 'Launch angle: ' + '{:1.0f}'.format(sl2.value) + ' degrees') 

 
#### Launch by clicking on animation
scene.bind('click', runanim)

##################################################################################################################################################################################

<IPython.core.display.Javascript object>